In [2]:
import copy
import glob

import cv2
import tqdm
import insightface
import numpy as np
from torchvision import transforms
import pandas as pd

from src.privacy_mechanisms.dtheta_privacy_mechanism import DThetaPrivacyMechanism
from src.utils import (
    img_tensor_to_cv2,
    load_insightface_models,
)


In [3]:
detect_model, recog_model = load_insightface_models()


def load_image(img_path, size=256):
    img = cv2.imread(img_path)

    try:
        bboxes, kpss = detect_model.detect(img)
        img = insightface.utils.face_align.norm_crop(
            img, landmark=kpss[0], image_size=size
        )
    except:
        img = cv2.resize(img, (size, size))

    return img


def generate_image(img, mechanism):
    img = copy.copy(img)
    img = transforms.ToTensor()(img)

    result = mechanism.process(img[None, :])
    gen_img = img_tensor_to_cv2(result[0, :])

    return gen_img

Loading face detection model.
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'has_user_compute_stream': '0', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_external_alloc': '0', 'gpu_mem_limit': '18446744073709551615', 'enable_cuda_graph': '0', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'cudnn_conv_use_max_workspace': '1', 'tunable_op_enable': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'prefer_nhwc': '0', 'use_ep_level_unified_stream': '0'}, 'CPUExecutionProvider': {}}
Loading facial recognition model.
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'has_user_compute_stream': '0', 'cud

In [8]:
mechanism = DThetaPrivacyMechanism(theta=0, epsilon=0)
mechanism.pad_ratio = 0

img_tot = []

for i in glob.glob("example_faces/*.png"):
    img = load_image(i, size=512)

    img_row = []

    for eps in [1, 10, 100, 1000]:
        mechanism.epsilon = eps
        priv_img = generate_image(img, mechanism)
        img_row.append(priv_img)

    img_row.append(img)

    mechanism.epsilon = -1

    for theta in [45, 90, 135, 180]:
        mechanism.theta = theta
        priv_img = generate_image(img, mechanism)
        img_row.append(priv_img)

    img_tot.append(np.concatenate(img_row, axis=1))


cv2.imwrite("figures/dtheta_simswap.jpg", np.concatenate(img_tot, axis=0))


eps rot: 91.58638117740252
theta rot: 91.58638117740252
eps rot: 91.36215233208391
theta rot: 91.36215233208391
eps rot: 79.45536341375445
theta rot: 79.45536341375445
eps rot: 38.15845482442246
theta rot: 38.15845482442246
eps rot: nan
theta rot: 44.99999249575457
eps rot: nan
theta rot: 89.99999999999999
eps rot: nan
theta rot: 135.00000750424547
eps rot: nan
theta rot: nan
eps rot: 87.93683966153516
theta rot: 92.06316033846485
eps rot: 85.30785582184198
theta rot: 94.69214417815802
eps rot: 78.25311255357381
theta rot: 101.74688744642619
eps rot: 39.12601300678924
theta rot: 140.87398699321076
eps rot: 0.0
theta rot: 45.000004074460655
eps rot: 0.0
theta rot: 90.00000000000001
eps rot: 0.0
theta rot: 134.99999592553937
eps rot: 0.0
theta rot: 179.97839215841148
eps rot: 91.72171732084082
theta rot: 88.27828267915919
eps rot: 92.73474307333541
theta rot: 87.2652569266646
eps rot: 82.18966075301884
theta rot: 97.81033924698116
eps rot: 39.2440495736197
theta rot: 140.75595042638034
e

True

In [18]:
DThetaPrivacyMechanism(theta=0, epsilon=-1)
mechanism.pad_ratio = 0

img_tot = []

faces = glob.glob("example_faces/*.png")

for i in [0, 3, 2, 5]:
    img = load_image(faces[i], size=512)

    img_row = [img]

    for theta in [30, 45, 60, 75, 90, 105, 120, 135]:
        np.random.seed(1)

        mechanism.theta = theta
        priv_img = generate_image(img, mechanism)
        img_row.append(priv_img)

    img_tot.append(np.concatenate(img_row, axis=1))


cv2.imwrite("figures/theta_slider.jpg", np.concatenate(img_tot, axis=0))


eps rot: nan
theta rot: 29.999987002263943
eps rot: nan
theta rot: 44.999992495754555
eps rot: nan
theta rot: 59.99999566742206
eps rot: nan
theta rot: 74.99999798924362
eps rot: nan
theta rot: 90.0
eps rot: nan
theta rot: 105.0000020107564
eps rot: nan
theta rot: 120.00000433257792
eps rot: nan
theta rot: 135.00000750424545
eps rot: 0.0
theta rot: 30.00000041915027
eps rot: 0.0
theta rot: 45.000000241996524
eps rot: 0.0
theta rot: 60.00000013971676
eps rot: 0.0
theta rot: 75.00000006484278
eps rot: 0.0
theta rot: 90.0
eps rot: 0.0
theta rot: 104.99999993515722
eps rot: 0.0
theta rot: 119.99999986028324
eps rot: 0.0
theta rot: 134.99999975800347
eps rot: 0.0
theta rot: 30.00000287529233
eps rot: 0.0
theta rot: 45.000001660050856
eps rot: 0.0
theta rot: 60.00000095843081
eps rot: 0.0
theta rot: 75.00000044480929
eps rot: 0.0
theta rot: 89.99999999999999
eps rot: 0.0
theta rot: 104.99999955519071
eps rot: 0.0
theta rot: 119.99999904156917
eps rot: 0.0
theta rot: 134.99999833994912
eps ro

True

In [36]:
from src.privacy_mechanisms.identity_dp_mechanism import IdentityDPMechanism
from src.privacy_mechanisms.metric_privacy_mechanism import MetricPrivacyMechanism
from src.privacy_mechanisms.pixel_dp_mechanism import PixelDPMechanism

mechanisms = []
# mechanisms.append(DThetaPrivacyMechanism(theta=90, epsilon=-1))
# mechanisms[-1].pad_ratio = 0
# mechanisms.append(DThetaPrivacyMechanism(theta=0, epsilon=1))
# mechanisms[-1].pad_ratio = 0
mechanisms.append("eps")
mechanisms.append("theta")
mechanisms.append(PixelDPMechanism(epsilon=5, b=4))
mechanisms.append(MetricPrivacyMechanism(epsilon=5, k=8))
mechanisms.append(IdentityDPMechanism(epsilon=1))
mechanisms[-1].pad_ratio = 0

img_labels = ["00004", "00018", "00039", "00114", "00145", "00418"]

img_tot = []

faces = glob.glob("example_faces/*.png")

for i in [1, 2, 4, 5]:
    img = load_image(faces[i], size=512)

    img_row = [img]

    for mech in mechanisms:
        if mech == "eps":
            priv_img = cv2.imread(
                f"D://github/anonghost/results/{img_labels[i]}_eps1.0_theta0.0.jpg"
            )
            priv_img = cv2.resize(priv_img, (512, 512))
        elif mech == "theta":
            priv_img = cv2.imread(
                f"D://github/anonghost/results/{img_labels[i]}_eps-1.0_theta90.0.jpg"
            )
            priv_img = cv2.resize(priv_img, (512, 512))
        else:
            priv_img = generate_image(img, mech)
        img_row.append(priv_img)

    img_tot.append(np.concatenate(img_row, axis=0))


cv2.imwrite("figures/baseline_comp.jpg", np.concatenate(img_tot, axis=1))


True

In [11]:
# anonghost figs
faces = glob.glob("example_faces/*.png")


img_tot = []

img_labels = ["00004", "00018", "00039", "00114", "00145", "00418"]


for f in [1, 5, 0, 2, 3, 4]:
    img = load_image(faces[f], size=512)

    img_row = [img]

    for theta in [30, 60, 90, 120, 150]:
        priv_img = cv2.imread(
            f"D://github/anonghost/results/{img_labels[f]}_eps-1.0_theta{theta}.0.jpg"
        )
        priv_img = cv2.resize(priv_img, (512, 512))
        img_row.append(priv_img)

    img_tot.append(np.concatenate(img_row, axis=1))


cv2.imwrite("figures/anonghost_slider.jpg", np.concatenate(img_tot, axis=0))

img_tot = []

for f in [0, 1, 2, 3, 4, 5]:
    img = load_image(faces[f], size=512)

    img_row = []

    for eps in [1, 10, 100, 1000]:
        priv_img = cv2.imread(
            f"D://github/anonghost/results/{img_labels[f]}_eps{eps}.0_theta0.0.jpg"
        )
        priv_img = cv2.resize(priv_img, (512, 512))
        img_row.append(priv_img)

    img_row.append(img)

    for theta in [45, 90, 135, 180]:
        priv_img = cv2.imread(
            f"D://github/anonghost/results/{img_labels[f]}_eps-1.0_theta{theta}.0.jpg"
        )
        priv_img = cv2.resize(priv_img, (512, 512))
        img_row.append(priv_img)

    img_tot.append(np.concatenate(img_row, axis=1))


cv2.imwrite("figures/method_anonghost.jpg", np.concatenate(img_tot, axis=0))


True

In [16]:
df = pd.read_csv("Results/Privacy/rank_k/CelebA_test_dtheta_privacy_theta0.0_eps0.0.csv")

matches_df = df.loc[df['k'] == 0]

true_images, anon_images = [], []

for i, row in tqdm.tqdm(matches_df.iterrows()):

    if len(true_images) > 100:
        break

    query_key = row["query_key"]
    img_label = query_key.split("___")[1]

    anon_image = f"Anonymized Datasets/CelebA_test_dtheta_privacy_theta0.0_eps0.0/Img/img_align_celeba/img_align_celeba/{img_label}.jpg"
    anon_image = cv2.imread(anon_image)
    anon_images.append(anon_image)
    true_image = f"Datasets/CelebA_test/Img/img_align_celeba/img_align_celeba/{img_label}.jpg"
    true_image = cv2.imread(true_image)
    true_images.append(true_image)


true_images1 = np.concatenate(true_images[0:8], axis=1)
anon_images1 = np.concatenate(anon_images[0:8], axis=1)
true_images2 = np.concatenate(true_images[8:16], axis=1)
anon_images2 = np.concatenate(anon_images[8:16], axis=1)
true_images3 = np.concatenate(true_images[16:24], axis=1)
anon_images3 = np.concatenate(anon_images[16:24], axis=1)

cv2.imwrite("figures/failure_cases_simswap.jpg", np.concatenate([true_images1, anon_images1, true_images2, anon_images2, true_images3, anon_images3], axis=0))


0it [00:00, ?it/s]

101it [00:00, 537.23it/s]


True

In [20]:
from src.dataset.celeba_identity_lookup import CelebAIdentityLookup
idlookup = CelebAIdentityLookup(identity_file_path="Datasets//CelebA//Anno//identity_CelebA.txt")

df = pd.read_csv("Results/Privacy/rank_k/CelebA_test_dtheta_privacy_theta0.0_eps0.0.csv")

all_ids = []
for i, row in df.iterrows():
    all_ids.append(idlookup.lookup(row["query_key"]))

matches_df = df.loc[df['k'] == 0]

num_sames = []

for i, row in tqdm.tqdm(matches_df.iterrows()):

    query_key = row["query_key"]
    this_id = idlookup.lookup(query_key)



    num_same = 0
    for aid in all_ids:
        if this_id == aid:
            num_same += 1
    num_sames.append(num_same)

print(np.mean(num_sames))


Building an identity lookup table for CelebA.


100%|██████████| 202599/202599 [00:00<00:00, 1595260.55it/s]
2902it [00:04, 599.62it/s]

23.50206753962784
